En este NoteBook se encuentra desarrolado el ETL (Extraction, Transformation and Load, en español, Extraccion, Transformacion y Carga) con los Datasets brindados por Henry 'movies_dataset' y 'credits'.
Posteriormente tambien desarrollo el EDA (Exploratory Data Analysis) del dataset que se fue desarrollando con el proceso de ETL ya limpiado y unido para un mejor tratamiento de los datos.  

## ETL Movies

In [29]:
import pandas as pd
import ast  # Para convertir el string del diccionario en un diccionario real
import numpy as np
from pandas import json_normalize
import random

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [30]:
#Leemos el archivo
data = pd.read_csv('/Users/julianscarpeccio/Desktop/Data Science HENRY/LABS/PI MLOps/movies_dataset.csv')

/var/folders/rl/4_3j3kb90d74wbdrz3mdbvn00000gn/T/ipykernel_94863/3564248637.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/julianscarpeccio/Desktop/Data Science HENRY/LABS/PI MLOps/movies_dataset.csv')


In [31]:
#Convertimos en Data Frame
df = pd.DataFrame(data)

In [32]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [33]:
# Funcion para desanidar columnas que son diccionarios dentro del Data Frame
def extraer_collection_names(collection):
    if pd.isnull(collection):
        return np.nan
    else:
        try:
            collection_dict = ast.literal_eval(collection)
            if isinstance(collection_dict, dict) and 'name' in collection_dict:
                return [collection_dict['name']]
            elif isinstance(collection_dict, list):
                names = [item['name'] for item in collection_dict if isinstance(item, dict) and 'name' in item]
                return names
            else:
                return np.nan
        except (ValueError, TypeError):
            return np.nan

In [34]:
# Luego aplicamos la función a 'belongs_to_collection'
df['belongs_to_collection'] = df['belongs_to_collection'].apply(extraer_collection_names)
# Luego aplicamos la función a 'genres'
df['genres'] = df['genres'].apply(extraer_collection_names)
# Luego aplicamos la función a 'production_companies'
df['production_companies'] = df['production_companies'].apply(extraer_collection_names)
# Luego aplicamos la función a 'production_countries'
df['production_countries'] = df['production_countries'].apply(extraer_collection_names)
# Luego aplicamos la función a 'spoken_languages'
df['spoken_languages'] = df['spoken_languages'].apply(extraer_collection_names)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [36]:
#llenamos valores con 0 como lo piden las consignas
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)

#eliminamos las filas con valores nulos en la columna 'release date'
df.dropna(subset=['release_date'], inplace=True)
#reemplazamos valores no convertibles con NaN, con la opción errors='coerce' se utiliza para indicar que si un valor no puede convertirse a fecha, se reemplace con NaN. 
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
#eliminamos filas con valores no convertibles
df.dropna(subset=['release_date'], inplace=True)

#creamos columna 'release_year'
df['release_year'] = df['release_date'].dt.year

#BORRAMOS DATOS QUE NO SIRVE DE LA COLUMNA BUDGET
#convierte la columna a valores numéricos y convierte los no numéricos en NaN
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
#Elimina las filas que contienen NaN en la columna "budget"
df.dropna(subset=['budget'], inplace=True)
#Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
#cuando no hay datos disponibles para calcularlo, toma el valor 0.
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['revenue'] != 0 and row['budget'] != 0 else 0, axis=1)


In [37]:
# Eliminamos columnas que no vamos a utilizar
# Lista de columnas a eliminar
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
# Eliminar las columnas del DataFrame
df = df.drop(columns=columnas_a_eliminar)

In [38]:
#Extraemos los nombres de 'belongs_to_collection' para extraerlos de la lista

def extract_name_from_list(lst):
    if isinstance(lst, list) and len(lst) > 0:
        return lst[0]  # Extraer el primer elemento de la lista
    else:
        return None

# Aplicar la función a la columna "collection" para extraer los nombres
df['belongs_to_collection'] = df['belongs_to_collection'].apply(extract_name_from_list)

In [39]:
#vista previa del data frame
df

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,None,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,Grumpy Old Men Collection,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000
3,None,16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760
4,Father of the Bride Collection,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,None,0,"[Drama, Action, Romance]",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[Westdeutscher Rundfunk (WDR), Working Title F...","[Canada, Germany, United Kingdom, United State...",1991-05-13,0.0,104.0,[English],Released,NaN,Robin Hood,5.7,26.0,1991,0.000000
45462,None,0,[Drama],111109,tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,0.0,360.0,[],Released,NaN,Century of Birthing,9.0,3.0,2011,0.000000
45463,None,0,"[Action, Drama, Thriller]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,0.0,90.0,[English],Released,A deadly game of wits.,Betrayal,3.8,6.0,2003,0.000000
45464,None,0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,0.0,0.0,1917,0.000000


## ETL Credits

In [40]:
#levantamos el otro data set brindado por Henry llamado 'credits.csv' y el creado anteriormente el cual llamé 'mi_dm'
# El archivo de Credits no se encuentra en el Repo por su tamaño, por favor en caso de necesitarlo encontrarlo en el siguiente link https://drive.google.com/file/d/1HxSWoxncj9Waqn2k5BJdeRaHLuZ4-MoE/view?usp=share_link

credits = pd.read_csv('/Users/julianscarpeccio/Desktop/Data Science HENRY/LABS/PI MLOps/credits.csv')
credits = pd.DataFrame(credits)

In [41]:
#Cambaimos el tipo de dato para la columna 'id' en ambos data frames
credits['id'] = credits['id'].astype('int')
df['id'] = df['id'].astype('int')
#unimos los data frames tomando el 'id' de la pelicula, columna que se encuentra en ambos.
df = df.merge(credits, on='id')

In [42]:
#Aplicamos la función ast.literal_eval a cada elemento de la columna "cast" y "crew" para convertir 
#una cadena que contiene una lista o diccionario en una lista de Python real facilitando el acceso y la manipulación 
#de los datos contenidos en esas columnas.

df['cast'] = df['cast'].apply(ast.literal_eval)
df['crew'] = df['crew'].apply(ast.literal_eval)

In [43]:
#La función itera a través de la lista de diccionarios y verifica si el valor asociado a la clave 'job' 
#en el diccionario actual es igual a 'Director'. Si encuentra un diccionario en el que el valor de 'job' sea 'Director', 
#la función devuelve el valor asociado a la clave 'name' en ese diccionario, que sería el nombre del director.

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [44]:
#aplicamos la funcion anterior a la columna 'crew'

df['director'] = df['crew'].apply(get_director)

In [45]:
#Creamos una funcion que extraiga los nombres de los actores de la columna 'cast'.

df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['cast'] = df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [46]:
#eliminamos la columna crew ya desanidada.
df.drop(columns='crew', inplace=True)

In [47]:
#creamos DF con las columnas que necesitaremos para el modelo de ML
df_ml = pd.DataFrame(df, columns=['tagline', 'overview', 'title'])
df_ml = df_ml.head(5000)
df_ml.to_csv('mi_df_ml', index=False)


## Machine Learning

Desde aquí, comenzaremos con el modelo de ML, vamos a construir un motor que calcule la similitud entre películas en función de ciertas métricas y sugiera películas que sean más similares a la pelicula que se ingrese. 

Tomaremos dos columnas del Dataframe:
**Overview y Tagline**

Segun el diccionario brindado por la acdemia:
**Overview** hace referencia a un pequeño resumen de la película
**Tagline** hace referencia a la frase celebre asociadaa la pelicula

Además, utilizaré un dataset mas pequño que el origianl debido a la limitación de potencia de cálculo disponible en render.

### Sistema de Recomendación Basado en la Descripción de Películas

Intentamos construir un recomendador utilizando el resumen y las frases celebres de las películas. Como no tenemos una métrica cuantitativa para juzgar el rendimiento, tendrá que hacerse cualitativamente.

In [48]:
# llenamos los valores faltantes (NaN) en la columna 'tagline'
# concatenamos el contenido de la columna 'overview' con el contenido de la columna 'tagline' 
# y la almacenamos en la columna 'description'.

df_ml['tagline'] = df_ml['tagline'].fillna('')
df_ml['description'] = df_ml['overview'] + df_ml['tagline']
df_ml['description'] = df_ml['description'].fillna('')

Aqui en el siguiente código que se utiliza la clase TfidfVectorizer de la biblioteca scikit-learn para realizar una transformación de texto en una matriz TF-IDF.
Se realiza una transformación de texto en una matriz numérica que representa la importancia relativa de las palabras y n-gramas en una colección de textos (en este caso, la columna 'description' del DataFrame 'df_ml'). Esta matriz se va a utlizizar posteriormente como entrada para un algoritmo de aprendizaje automático.

In [49]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.01, stop_words='english')
tfidf_matrix = tf.fit_transform(df_ml['description'])

La matriz TF-IDF resultante es una representación numérica de las palabras en el texto, donde las filas corresponden a documentos y las columnas corresponden a términos (palabras y n-gramas). Los valores en la matriz representan el peso TF-IDF de cada término en cada documento. El TF-IDF pondera la frecuencia de un término en un documento en relación con su frecuencia en todo el corpus de documentos, lo que ayuda a destacar términos más importantes para cada documento.

In [50]:
tfidf_matrix.shape

(5000, 489)

### Similitud del Coseno

Utilizaré la Similitud del Coseno para calcular una cantidad numérica que determine la similitud entre dos películas.
Dado que hemos utilizado el TF-IDF Vectorizer, el cálculo del Producto nos dará directamente el Puntaje de Similitud del Coseno.
Por lo tanto, utilizaremos la función linear_kernel de sklearn en lugar de cosine_similarities, ya que es mucho más rápida.
Ahora tenemos una matriz de similitud del coseno entre pares de todas las películas en nuestro conjunto de datos. El siguiente paso es escribir una función que devuelva las 5 películas más similares en función del puntaje de similitud del coseno.

In [51]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [52]:
cosine_sim[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [53]:
df_ml = df_ml.reset_index()
titles = df_ml['title']
indices = pd.Series(df_ml.index, index=df_ml['title'])

In [54]:
#Creamos la funcion de Recomendacion que nos servirá de base para emular la que usaremos en la API.

def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [55]:
#probamos la función

get_recommendations('Apollo 13').head(5)

4105                     The Dish
3643                    Communion
133                Down Periscope
4248    Atlantis: The Lost Empire
3245              Mission to Mars
Name: title, dtype: object